# python3.5の仮想環境を構築してpygameを試す
pygameは、現時点でPython 3.5の環境じゃないとインストールが出来ないので、仮想環境を作ってpygameを試す。
  - pygameが動くpython 3.5の仮想環境を構築する。 `>conda create -n py35 python=3.5`
  - pygameをインストールする。 `>conda install -c cogsci pygame`
  - プログラムを試す。

## 1. python3.5の仮想環境構築 (Anaconda Promptで実行)
### 1-1. 現在の環境を確認
baseの環境のみが存在している。
~~~
(base) C:\***>conda info -e
# conda environments:
base                  *  C:\***\anaconda3
~~~

### 1-2. py35という名前で仮想環境を作成
現時点でpygameがpython3.5に対応しているため、python3.5をインストールする。
~~~
(base) C:\***>conda create -n py35 python=3.5
~~~

### 1-3. 仮想環境が出来たか確認
py35という名前の仮想環境が出来ている。\*が付いた方がactiveな環境。
~~~
(base) C:\***>conda info -e
# conda environments:
base                  *  C:\***\anaconda3
py35                     C:\***\anaconda3\envs\py35
~~~

### 1-4. 仮想環境を切り替え
py35をactiveにする。
~~~
(base) C:\***>activate py35
base                     C:\***\anaconda3
py35                  *  C:\***\anaconda3\envs\py35
~~~

### 1-5. py35環境にJupyterとSpyderをインストールする
~~~
(py35) C:\***>conda install jupyter
(py35) C:\***>conda install spyder
~~~

### 1-6. 環境をアップデートしないとうまく動かなかったので、アップデートする。

~~~
>conda update –-all
~~~

### py35の仮想環境を削除する場合。
一旦対象をdeactivate。（baseをacctivate）し、ターミナルを再起動する。その後、py35の削除処理を行う。
~~~
(py35) C:\***>conda.bat activate base
(base) C:\***>conda remove -n py35 --all
~~~

## 2. pygameのサンプルプログラムを試す
ウィンドウ内をクリックすると赤い粒（Dot）が生成され、ウィンドウ内を有機的に動き回るサンプルプログラム。

In [1]:
import pygame
from pygame.locals import MOUSEBUTTONDOWN, QUIT
import sys
import random
import math

## 2-1. Dotクラス生成

In [2]:
class Dot:
    
    # Initialize the object.
    def __init__(self, x, y, vx, vy):
        # position coordinates.
        self.x, self.y = x, y
        # velocity.
        self.vx, self.vy = vx, vy

    # When the dot reached each edge of the window, 
    # reverse the direction of velocity.
    def update(self, WINDOW_W, WINDOW_H):
        self.x += self.vx
        self.y += self.vy
        
        # Right edge.
        if self.x > WINDOW_W:
            self.vx, self.x = -self.vx, WINDOW_W
            return
        
        # Left edge.
        if self.x < 0:
            self.vx, self.x = -self.vx, 0
            return
        
        # Upper edge.
        if self.y > WINDOW_H:
            self.vy, self.y = -self.vy, WINDOW_H
            return

        # Lower edge.
        if self.y < 0:
            self.vy, self.y = -self.vy, 0
            return

    # Draw a dot on the screen.
    # `circle(surface, color, center, radius)`
    # `circle(surface, color, center, radius, width=0)`
    def draw(self, screen):
        x, y = int(self.x), int(self.y)
        pygame.draw.circle(screen, (255,0,0), (x,y), 5, 1)
    
    # separation: dot to avoid crowding local flockmates
    # r_s : The radius of the circle of private area. 
    def separation(self, r_s):
        tvx = tvy = cnt = 0
        
        # If the distance to other dots is too short, when it is less than r_s, 
        # calculate a unit vector in the direction of repulsion and add.
        # d[1] : The distance from myself to other dots.
        #        Don't calculate in case d[1]=0 to avoide error occard.
        for d in self.others:
            if d[1] < r_s and d[1] != 0:
                tvx -= (d[0].x - self.x) / d[1]
                tvy -= (d[0].y - self.y) / d[1]
                cnt += 1
                
        # Calculate an average of the unit vector by divide the sum of a unit vector by a number of it.
        if cnt != 0:
            self.vx_s, self.vy_s = tvx / cnt, tvy / cnt
        else:
            self.vx_s, self.vy_s = self.vx, self.vy
    
    # alignment: dot towards the average heading of local flockmates.
    # r_a : The radius of the circle to judge whether to align or not.
    def alignment(self, r_a):
        tvx = tvy = cnt = 0
        
        # Sum all velocity of dots that are inside of a circle with radius "r_a".
        # d[1] : The distance from myself to other dots.
        for a in self.others:
            if a[1] < r_a:
                tvx += a[0].vx
                tvy += a[0].vy
                cnt += 1
                
        # Calculate an average of the unit vector by divide the sum of a unit vector by a number of it.
        if cnt != 0:
            self.vx_a, self.vy_a = tvx / cnt, tvy / cnt
        else:
            self.vx_a, self.vy_a = self.vx, self.vy
        
    #cohesion: dot to move towards the average position (center of mass) of local flockmates   
    # r_a : The radius of the circle to judge whether the other dots are the same flockmate or not.
    def cohesion(self, r_c):
        tx = ty = cnt = 0
        
        # Sum all x and y coordinate of dots that are inside of a circle with radius "r_c".
        # d[1] : The distance from myself to other dots.
        for a in self.others:
            if a[1] < r_c:
                tx += a[0].x
                ty += a[0].y
                cnt += 1
        
        # Calucurate the coordiate ot the center of gravity.
        if cnt != 0:
            tx, ty = tx / cnt, ty / cnt
            # Calculate the distance from itself to the center of gravity.
            d = math.sqrt((tx - self.x)**2 + (ty - self.y)**2)
            
            # Calculate he unit vector.
            if d != 0:
                self.vx_c = (tx - self.x) / d
                self.vy_c = (ty - self.y) / d
        else:
            self.vx_c, self.vy_c = self.vx, self.vy
            
    def rule(self, agent_list, r_s, r_a, r_c):
        #self.others = tuple([(a, math.sqrt((a.x - self.x)**2 \
        #                    + (a.y - self.y)**2)) \
        #                     for a in agent_list if a != self])
        # If the above program is rewritten, it will be as follows.
        tpls = ()
        for a in agent_list:
            if a != self:
                dist= math.sqrt((a.x - self.x)**2 + (a.y - self.y)**2)
                tpl = tuple([(a, dist)])
                tpls = tpls + tpl                
        self.others = tpls
        
        if len(self.others) < 1: return
        
        self.separation(r_s)
        self.alignment(r_a)
        self.cohesion(r_c)
        
        tvx = self.vx_s * 1.0 + self.vx_a * 0.4 + self.vx_c * 0.2
        tvy = self.vy_s * 1.0 + self.vy_a * 0.4 + self.vy_c * 0.2
        
        n = math.sqrt(tvx**2 + tvy**2)
        
        # Set the new velocity.
        self.vx, self.vy = 2 * tvx / n, 2 * tvy / n

## 2-2. メイン処理

In [3]:
# Initialise screen
pygame.init()
WINDOW_W, WINDOW_H = 800, 600
screen = pygame.display.set_mode((WINDOW_W, WINDOW_H))
pygame.display.set_caption('Basic Pygame program')

# Create the font object.
# `.font.Font(which font to use, the size of the font)`
font = pygame.font.Font(None, 28)

clock = pygame.time.Clock()
dot_list = []
rule_on = 1

while True:
    # Set framerate. (FPS : frames per second.)
    clock.tick(60)
    
    # Fill / refill screen with white.
    screen.fill((255, 255, 255))
    
    # Process each dot.
    for d in dot_list:
        if rule_on == 1:
            d.rule(dot_list, 15, 30, 50)

        d.update(WINDOW_W, WINDOW_H)
        d.draw(screen)

    # Show the number of the dots and whether the rule is being applied or not.
    # `.render(the text to be rendered, 
    #          whether or not it should be anti-aliased (1=yes, 0=no),
    #          the color of the text )`
    s1 = "dots : " + str(len(dot_list))
    if rule_on == 1:
        s2 = "the rule is being applied."
        text = font.render(s1 + " / " + s2, True, (255,0,0), (255,239,213))
    else:
        s2 = "the rule is off."
        text = font.render(s1 + " / " + s2, True, (60,179,113))


    # blit (bitblt : bit block transfer)
    screen.blit(text, (1, 1))

    # Get an event.
    for e in pygame.event.get():

        # Create some dots at the mouse position when
        # the mouse wheel is turned forward or turned back.
        if e.type == MOUSEBUTTONDOWN and ( e.button == 4 or e.button == 5 ):

            # Set velocity by use a random number.
            # `numpy.random.uniform(low=0.0, high=1.0, size=None)`
            vx = random.uniform(-15, 15)
            vy = random.uniform(-15, 15)

            # Create a dot.
            dot = Dot(e.pos[0], e.pos[1], vx, vy)

            # Add created dot into the list.
            dot_list.append(dot)

        # Change the rule when the left button is pressed.
        if e.type == MOUSEBUTTONDOWN and e.button == 1:
            rule_on = 1 - rule_on

        # Stop the application when 'close' button of
        # the window is pressed.
        if e.type == QUIT:
            pygame.quit()
            sys.exit()

    pygame.display.update()

ZeroDivisionError: float division by zero